In [1]:
import sys
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)



In [2]:
def adjust_sensors_delay_time(implementation):
    """
    This function returns the delay time for each sensor in order to adjust the data
    """
    #esp32
    seconds_delay = 0
    milliseconds_delay = 0
    if implementation == "ESP32S3": 
        seconds_delay = 2
        milliseconds_delay = 920

    return timedelta(seconds=seconds_delay, milliseconds=milliseconds_delay)

In [3]:
def jobs_unix_timestamp(implementation, day, month):
    epoch = datetime.utcfromtimestamp(0)

    startDT = datetime(2023, month, 28, 9, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startTS = int((startDT - epoch).total_seconds() * 1000.0)
    endDT = datetime(2023, month, day, 18, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endTS = int((endDT - epoch).total_seconds() * 1000.0)
    

    return str(startTS), str(endTS)

In [4]:
RFID_analysis_keys = [
    'gabarit'
]

TASK_analysis_keys = [
    'task', 'empid'
]


In [5]:
def json_csv(data, dataset, implementation):
    data_df = pd.DataFrame()
    first_flag = True
    
    if dataset == "rfid":
        analysis_keys = RFID_analysis_keys
    elif dataset == "task":
        analysis_keys = TASK_analysis_keys



    for analysis_key in analysis_keys:
        if first_flag:
            data_df = pd.DataFrame(data[analysis_key])
            data_df = data_df.rename(columns={'value': analysis_key})
            first_flag = False
        else:
            data_df = data_df.join(pd.DataFrame(data[analysis_key]).set_index('ts'), on='ts')
            data_df = data_df.rename(columns={'value': analysis_key})
    # change the timestamp to seconds
    data_df['ts'] = pd.to_datetime(data_df['ts']/1000,unit='s') + timedelta(hours=1) - adjust_sensors_delay_time(implementation)
    data_df['ts']= data_df['ts'].astype('datetime64[s]')
    return data_df.iloc[::-1]

def getToken():
    username = 'isabel.portugal@tecnico.ulisboa.pt' #email used to create your ThingsBoard account (your email associated with tennant)
    password = 'tHn%Mv1pnmq2av='		#password of your ThingsBoard account (or your tennant account)
    url = 'http://iot.istartlab.tecnico.ulisboa.pt/api/auth/login'
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    loginJSON = {'username': username, 'password': password}
    tokenAuthResp = requests.post(url, headers=headers, json=loginJSON).json()
    #print(tokenAuthResp)
    token = tokenAuthResp['token']
    return token

In [6]:
tkn = getToken()

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)
#url to test connection
url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/27110c40-2a6b-11ee-9b43-67453a5ac630/keys/timeseries"

response = requests.get(url, headers=my_headers)
print(response) , print(response.text)

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)

<Response [404]>
Device with requested id wasn't found!


In [7]:
# CHANGE ONLY THIS
month_int = 9
day_int = 29
dataset = "rfid" #"task"

if month_int < 10:
    month_str = "0" + str(month_int)
else:
    month_str = str(month_int)
day_str = str(day_int)

startTS, endTS = jobs_unix_timestamp("ESP32S3", day_int, month_int)

limit = "1000000000"

date_filename = dataset + "_" + day_str + "_" + month_str

devices = ["27110c40-2a6b-11ee-9b43-67453a5ac633","f4765800-2cc5-11ee-a22d-b5d6c11ab119","10fc9700-2cc6-11ee-a22d-b5d6c11ab119"]
cnt = 1
#keys = "ts,"

for analysis_key in RFID_analysis_keys:
    keys = "ts,gabarit"

#for analysis_key in TASK_analysis_keys:
#    keys = "ts,task,empid"  

print(keys)

for dev in devices:
    url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/" + dev + "/values/timeseries?keys=" + keys + "&startTs=" + startTS + "&endTs=" + endTS + "&limit=" + limit
    response = requests.get(url, headers=my_headers)
    print("response received", response.text)
    data = json.loads(response.text)
    data_df = json_csv(data, dataset, "ESP32S3")
    data_df.to_csv(date_filename + "_" + str(cnt) + ".csv")
    cnt += 1

print("acc done")


ts,gabarit
response received {"gabarit":[{"ts":1695907397240,"value":"0"},{"ts":1695907361848,"value":"9"},{"ts":1695907195968,"value":"0"},{"ts":1695907119088,"value":"14"},{"ts":1695907059468,"value":"0"},{"ts":1695906792480,"value":"11"},{"ts":1695906738548,"value":"0"},{"ts":1695906481328,"value":"20"},{"ts":1695906311346,"value":"0"},{"ts":1695906214108,"value":"10"},{"ts":1695906113529,"value":"0"},{"ts":1695902321060,"value":"0"},{"ts":1695902315399,"value":"6"},{"ts":1695902314908,"value":"6"},{"ts":1695902311239,"value":"6"},{"ts":1695902272588,"value":"0"},{"ts":1695902203261,"value":"16"},{"ts":1695902121527,"value":"0"},{"ts":1695902116387,"value":"15"},{"ts":1695902071388,"value":"0"},{"ts":1695902050240,"value":"21"},{"ts":1695901990619,"value":"0"},{"ts":1695901986068,"value":"19"},{"ts":1695901950988,"value":"0"},{"ts":1695901856827,"value":"17"},{"ts":1695901784200,"value":"0"},{"ts":1695901779379,"value":"3"},{"ts":1695901764318,"value":"0"},{"ts":1695901654376,"value

In [8]:
print(response.text)

{}
